There are three goals of this script:
1. To check the results of the mock community extraction and sequencing to look for any bias in the extraction and sequencing methods that may influence the data 
2. To use the 'blank' sample to remove or decontaminate the coral samples from any background bacterial contamination from the lab
3. Remove mitochondria, chloroplasts and unknown ASV's from the feature table

We're going to use the qiime2 plugin [quality-control](https://docs.qiime2.org/2023.9/plugins/available/quality-control/) to achieve these goals

This [qiime2 tutorial](https://docs.qiime2.org/2023.9/tutorials/quality-control/) will help

## Python 3 API import qiime plugins

In [3]:
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2.plugins import quality_control
import os
import shutil

# Evaluate mock community

[Zymo MIQ score](https://www.zymoresearch.com/blogs/blog/how-to-quantify-bias-with-mock-microbial-community-standards)

[ZymoBIOMICS Microbial Community Standard](https://www.zymoresearch.com/blogs/blog/zymobiomics-microbial-standards-optimize-your-microbiomics-workflow%20%20%20%20%20%20%20%20)

Analyzing 16S sequencing data of the [ZymoBIOMICS® Standard](https://www.zymoresearch.com/collections/zymobiomics-microbial-community-standards/products/zymobiomics-microbial-community-standard) When sequencing the ZymoBIOMICS® standards analyze them using your regular 16S analysis pipelines, such as Qiime and Mothur. You can compare the measured composition with the theoretical composition of the standard. Questions that should be kept in mind during this comparison include: (1) whether your measurement covers all strains with the proper taxonomy assignment and with correct abundance, (2) whether your measurement indicates the presence of foreign taxa with significant abundance. Taxonomy assignment might be incorrect or improper because of problems in the reference database. Abundance estimation might be off because of bias in DNA extraction, bias in library preparation, poor quality of MiSeq runs, etc. The presence of foreign taxa might indicate process contamination, poor sequencing quality, PCR chimera in library preparation, defects in bioinformatics analysis, defects in reference database, etc. Both the ZymoBIOMICS®Microbial Community Standard and the DNA Standard are certified to have low impurities levels, < 0.01% by DNA abundance. So any foreign taxa with abundance higher than 0.01% are derived from artifacts in your workflow.

Checkout this Zymo resource page on [How to quantify bias with mock microbial community standards](https://www.zymoresearch.com/blogs/blog/how-to-quantify-bias-with-mock-microbial-community-standards)

Measurement Integrity Quotient (MIQ) score
The MIQ score simplifies bias assessment by assigning a score from 0 to 100 and can be interpreted like a grade on a high school exam, with >90 being excellent, 80-89 being good, and so on (Figure 1). This is achieved by measuring the fidelity of measured relative abundances compared to a known input. The MIQ score is presented in a user-friendly report which also includes different plots to visually represent the evaluation. These include radar plots, taxa bar plots, and read fate counts

![example-shotgun-composition-MIQ-score](https://files.zymoresearch.com/images/blog_post-miq_score_figure-2b.png)

Called and asked them to send me the raw sequence data for the theoretical composition of ZymoBIOMICS Microbial Community Standard [(Cat#D6300)](https://www.zymoresearch.com/collections/zymobiomics-microbial-community-standards/products/zymobiomics-microbial-community-standard)
<br>
Theoretical Composition Based on Genomic DNA: 
- Listeria monocytogenes - 12% 
- Pseudomonas aeruginosa - 12% 
- Bacillus subtilis - 12% 
- Escherichia coli - 12% 
- Salmonella enterica - 12% 
- Lactobacillus fermentum - 12% 
- Enterococcus faecalis - 12% 
- Staphylococcus aureus - 12%
- Saccharomyces cerevisiae - 2%
- Cryptococcus neoformans - 2%

To do this we need:
- mock-expected.qza "FeatureTable[RelativeFrequency]"
- mock-observed.qza "FeatureTable[RelativeFrequency]"

In [4]:
# Specify the path to the "output" directory
output_dir = "../output"  # Replace with the actual path to your "output" directory

# Directory name
dir_name = "filtered"

# Full path to the "demux" directory in the "output" directory
dir_path = os.path.join(output_dir, dir_name)

# Check if the directory exists
if os.path.exists(dir_path) and os.path.isdir(dir_path):
    # If it exists, delete its contents
    print(f"Removing contents of {dir_name}...")
    for item in os.listdir(dir_path):
        item_path = os.path.join(dir_path, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
else:
    # If it doesn't exist, create an empty folder
    print(f"Creating {dir_name} in {output_dir}...")
    os.makedirs(dir_path)

Removing contents of filtered...


Exclude the mock community from the feature table for downstream analysis

In [5]:
!qiime feature-table filter-samples \
  --i-table ../output/dada2/table.qza \
  --m-metadata-file ../data/sample-metadata-verbose.tsv \
  --p-where "BarcodeSequence ='GATTAAGGTG'" \
  --p-exclude-ids true \
  --o-filtered-table ../output/filtered/table-no-mock.qza

Saved FeatureTable[Frequency] to: ../output/filtered/table-no-mock.qza


In [6]:
!qiime feature-table summarize \
  --i-table ../output/filtered/table-no-mock.qza \
  --o-visualization ../output/filtered/table-no-mock.qzv \
  --m-sample-metadata-file ../data/sample-metadata-verbose.tsv

Saved Visualization to: ../output/filtered/table-no-mock.qzv


In [7]:
Visualization.load('../output/filtered/table-no-mock.qzv')

<visualization: Visualization uuid: 5e6859c4-0c2d-4a68-9874-d4868a67372c>

# Decontaminate

Here I want to take all the features that were found in the 'blank' sample, and subtract them from the feature table. aka, getting rid of contamination! Yay! 

We first need to generate some files for input:
- query-seqs.qza "FeatureData[Sequence]"
  | We want this to be a list of the sequences found in the blank, the ones we want to remove!
- reference-seqs.qza "FeatureData[Sequence]"
  | This is the list of sequences found in the coral samples

We want to search for any sequences present in the query against the reference.
Any sequences from the blank found in the coral will be 'hits', any sequences from the corals that don't match those in the blank will be 'miss'

- query-table.qza "FeatureTable[Frequency]" | This will be all the coral samples

Figure out what the contamination is: https://docs.qiime2.org/2023.9/plugins/available/quality-control/decontam-identify/

(Metadata-based filtering)[https://docs.qiime2.org/2019.4/tutorials/filtering/#:~:text=view%20%7C%20download-,Metadata%2Dbased%20filtering,-%C2%B6]

Modify metadata to keep only the blank sample!

In [8]:
import pandas as pd

In [9]:
# read in tsv sample metadata as a csv
df = pd.read_csv('../data/sample-metadata-verbose.tsv', delimiter='\t')
# make a list of SampleID values to remove
keep = ['blank.']
# remove those rows 
df = df[df['#SampleID'].isin(keep)]
df

,#SampleID,BarcodeSequence,LinkerPrimerSequence,BarcodeName,ReversePrimer,ProjectName,Description,Pae,Temp,PaeTemp,Colony,Tank
20,blank.,TATGTGCAAT,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0093,GGACTACNVGGGTWTCTAAT,060823STillcus515F,blank.,blank,blank,blank,blank,blank


In [10]:
# save the new df as a tsv
df.to_csv('../data/sample-metadata-blank.tsv', sep='\t', index=False)

Create feature table of only the blank sample

In [11]:
!qiime feature-table filter-samples \
  --i-table ../output/dada2/table.qza \
  --m-metadata-file ../data/sample-metadata-blank.tsv \
  --o-filtered-table ../output/filtered/table-blank.qza

Saved FeatureTable[Frequency] to: ../output/filtered/table-blank.qza


 Alternatively, instead of using pandas, you can use `qiime feature-table filter-samples` :

In [13]:
!qiime feature-table filter-samples \
  --i-table ../output/dada2/table.qza \
  --m-metadata-file ../data/sample-metadata-verbose.tsv \
  --p-where "BarcodeSequence ='TATGTGCAAT'" \
  --o-filtered-table ../output/filtered/table-blank.qza

Saved FeatureTable[Frequency] to: ../output/filtered/table-blank.qza


In [14]:
!qiime feature-table summarize \
  --i-table ../output/filtered/table-blank.qza \
  --o-visualization ../output/filtered/table-blank.qzv \
  --m-sample-metadata-file ../data/sample-metadata-blank.tsv

Saved Visualization to: ../output/filtered/table-blank.qzv


In [15]:
Visualization.load('../output/filtered/table-blank.qzv')

<visualization: Visualization uuid: 5f2b6a6e-2add-4b85-ad74-c75167436ff5>

### Filter representative sequences 
Here I make an artifact file that contained only the sequences found in the blank sample.
This artifact creation uses the blank-filtered frequency table and the representative sequences `FeatureData[Sequence]` as input. 

In [16]:
!qiime feature-table filter-seqs \
  --i-data ../output/dada2/representative_sequences.qza \
  --i-table ../output/filtered/table-blank.qza \
  --o-filtered-data ../output/filtered/rep-seqs-blank.qza

Saved FeatureData[Sequence] to: ../output/filtered/rep-seqs-blank.qza


In [17]:
!qiime quality-control exclude-seqs \
  --i-query-sequences ../output/filtered/rep-seqs-blank.qza \
  --i-reference-sequences ../output/dada2/representative_sequences.qza \
  --p-method blast \
  --p-perc-identity 0.97 \
  --p-perc-query-aligned 0.97 \
  --o-sequence-hits ../output/filtered/hits.qza \
  --o-sequence-misses ../output/filtered/misses.qza

Saved FeatureData[Sequence] to: ../output/filtered/hits.qza
Saved FeatureData[Sequence] to: ../output/filtered/misses.qza


In [18]:
!qiime feature-table filter-features \
  --i-table ../output/filtered/table-no-mock.qza \
  --m-metadata-file ../output/filtered/hits.qza \
  --o-filtered-table ../output/filtered/table-no-mock-no-hits.qza \
  --p-exclude-ids

Saved FeatureTable[Frequency] to: ../output/filtered/table-no-mock-no-hits.qza


In [19]:
!qiime feature-table summarize \
  --i-table ../output/filtered/table-no-mock-no-hits.qza \
  --o-visualization ../output/filtered/table-no-mock-no-hits.qzv \
  --m-sample-metadata-file ../data/sample-metadata-verbose.tsv

Saved Visualization to: ../output/filtered/table-no-mock-no-hits.qzv


In [21]:
Visualization.load("../output/filtered/table-no-mock-no-hits.qzv")

<visualization: Visualization uuid: de3cb6d9-a5a3-47d6-9321-7e7bce537c1e>

# Remove Mitochondria, Chloroplastis & Unknowns & Methylophaga with Taxonomy-based filtering
[qiime2 tutorial](https://docs.qiime2.org/2019.4/tutorials/filtering/)

In [22]:
!qiime taxa filter-table \
  --i-table ../output/filtered/table-no-mock-no-hits.qza \
  --i-taxonomy ../output/taxonomy/classification.qza \
  --p-exclude mitochondria,chloroplast,unassigned,methylophaga \
  --o-filtered-table ../output/filtered/table-no-mock-no-hits-taxon-filtered.qza

Saved FeatureTable[Frequency] to: ../output/filtered/table-no-mock-no-hits-taxon-filtered.qza


Now let's view the taxonomy bar plot to confirm that mitochondria, chloroplasts, and unknown features were removed from the filtered feature table

In [23]:
!qiime taxa barplot \
    --i-table ../output/filtered/table-no-mock-no-hits-taxon-filtered.qza \
    --i-taxonomy ../output/taxonomy/classification.qza \
    --m-metadata-file ../data/sample-metadata-verbose.tsv \
    --o-visualization ../output/taxonomy/taxa_barplot-no-mock-no-hits-taxon-filtered.qzv

Saved Visualization to: ../output/taxonomy/taxa_barplot-no-mock-no-hits-taxon-filtered.qzv


In [24]:
Visualization.load("../output/taxonomy/taxa_barplot-no-mock-no-hits-taxon-filtered.qzv")

<visualization: Visualization uuid: eda3d8bc-99ca-4dfb-bb60-8d9da20431da>